In [1]:
import os
from tqdm import tqdm
import feather
import pandas as pd
import numpy as np
from scipy import stats
import gc
from joblib import Parallel, delayed

In [2]:
def append_to_csv(batch, csv_file):
    props = dict(encoding='utf-8', index=False)
    if not os.path.exists(csv_file):
        batch.to_csv(csv_file, **props)
    else:
        batch.to_csv(csv_file, mode='a', header=False, **props)

def delete_file_if_exists(filename):
    if os.path.exists(filename):
        os.remove(filename)

In [3]:
fn = '../cache/bkup/train10_t_store{}.csv'.format(1)
# fn = '../cache/bkup2/test7_1_t_store{}.csv'.format(1)
df =pd.read_csv(fn)

/home/watts/anaconda2/envs/sdp35/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
df.head()

,id,date,store_nbr,item_nbr,unit_sales,onpromotion,item_family,item_class,perishable,city,...,dow,doy,dcount,h_type,h_desc,pd,wbe,wae,wfe,store_item_nbr
0,578,2013-01-02,1,103665,2.0,Unk,BREAD/BAKERY,2712,1,Quito,...,2,2,734870,Work Day,Normal,0,172,0,172,1_103665
1,579,2013-01-02,1,105574,8.0,Unk,GROCERY I,1045,0,Quito,...,2,2,734870,Work Day,Normal,0,172,0,172,1_105574
2,580,2013-01-02,1,105575,15.0,Unk,GROCERY I,1045,0,Quito,...,2,2,734870,Work Day,Normal,0,172,0,172,1_105575
3,581,2013-01-02,1,105577,2.0,Unk,GROCERY I,1045,0,Quito,...,2,2,734870,Work Day,Normal,0,172,0,172,1_105577
4,582,2013-01-02,1,105737,2.0,Unk,GROCERY I,1044,0,Quito,...,2,2,734870,Work Day,Normal,0,172,0,172,1_105737


In [20]:
df.shape

(62416, 27)

In [21]:
len(set(df.item_nbr))

3901

In [22]:
np.min(df.date)

'2017-08-16'

In [23]:
np.max(df.date)

'2017-08-31'

In [24]:
np.datetime64(np.max(df.date)) - np.datetime64(np.min(df.date))

numpy.timedelta64(15,'D')

In [25]:
365 * 4

1460

In [3]:
def set_recent_stats(df2, date, days, mean, median, std, skew, kurtosis, ten_by_ninety):
    df2.loc[0,'date'] = date
    df2.loc[0,'us_{}_mean'.format(days)] = mean
    df2.loc[0,'us_{}_med'.format(days)] = median
    df2.loc[0,'us_{}_std'.format(days)] = std
    df2.loc[0,'us_{}_skew'.format(days)] = skew
    df2.loc[0,'us_{}_kurt'.format(days)] = kurtosis
    df2.loc[0,'us_{}_tbn'.format(days)] = ten_by_ninety


def set_recent_stats_promo(df2, date, days, mean, median, std, skew, kurtosis, ten_by_ninety):
    df2.loc[0,'date'] = date
    df2.loc[0,'us_{}_p_mean'.format(days)] = mean
    df2.loc[0,'us_{}_p_med'.format(days)] = median
    df2.loc[0,'us_{}_p_std'.format(days)] = std
    df2.loc[0,'us_{}_p_skew'.format(days)] = skew
    df2.loc[0,'us_{}_p_kurt'.format(days)] = kurtosis
    df2.loc[0,'us_{}_p_tbn'.format(days)] = ten_by_ninety


def set_recent_stats_promo2(df2, date, days, mean, median, std, skew, kurtosis, ten_by_ninety):
    
    df2.loc[0,'date'] = date
    df2.loc[0,'us_{}_p_mean'.format(days)] = mean
    df2.loc[0,'us_{}_p_med'.format(days)] = median
    df2.loc[0,'us_{}_p_std'.format(days)] = std
    df2.loc[0,'us_{}_p_skew'.format(days)] = skew
    df2.loc[0,'us_{}_p_kurt'.format(days)] = kurtosis
    df2.loc[0,'us_{}_p_tbn'.format(days)] = ten_by_ninety


def set_recent_stats_promo3(df2, date, days, mean, median, std, skew, kurtosis, ten_by_ninety):
    df2.loc[0,'date'] = date
    df2.loc[0,'us_{}_p_mean'.format(days)] = mean
    df2.loc[0,'us_{}_p_med'.format(days)] = median
    df2.loc[0,'us_{}_p_std'.format(days)] = std
    df2.loc[0,'us_{}_p_skew'.format(days)] = skew
    df2.loc[0,'us_{}_p_kurt'.format(days)] = kurtosis
    df2.loc[0,'us_{}_p_tbn'.format(days)] = ten_by_ninety

def set_recent_stats_hol(df2, date, days, mean, median, std, skew, kurtosis, ten_by_ninety):
    df2.loc[0,'date'] = date
    df2.loc[0,'us_{}_h_mean'.format(days)] = mean
    df2.loc[0,'us_{}_h_med'.format(days)] = median
    df2.loc[0,'us_{}_h_std'.format(days)] = std
    df2.loc[0,'us_{}_h_skew'.format(days)] = skew
    df2.loc[0,'us_{}_h_kurt'.format(days)] = kurtosis
    df2.loc[0,'us_{}_h_tbn'.format(days)] = ten_by_ninety


def set_recent_stats_per(df2, date, days, mean, median, std, skew, kurtosis, ten_by_ninety):
    df2.loc[0,'date'] = date
    df2.loc[0,'us_{}_per_mean'.format(days)] = mean
    df2.loc[0,'us_{}_per_med'.format(days)] = median
    df2.loc[0,'us_{}_per_std'.format(days)] = std
    df2.loc[0,'us_{}_per_skew'.format(days)] = skew
    df2.loc[0,'us_{}_per_kurt'.format(days)] = kurtosis
    df2.loc[0,'us_{}_per_tbn'.format(days)] = ten_by_ninety


def set_recent_stats_nper(df2, date, days, mean, median, std, skew, kurtosis, ten_by_ninety):
    df2.loc[0,'date'] = date
    df2.loc[0,'us_{}_nper_mean'.format(days)] = mean
    df2.loc[0,'us_{}_nper_med'.format(days)] = median
    df2.loc[0,'us_{}_nper_std'.format(days)] = std
    df2.loc[0,'us_{}_nper_skew'.format(days)] = skew
    df2.loc[0,'us_{}_nper_kurt'.format(days)] = kurtosis
    df2.loc[0,'us_{}_nper_tbn'.format(days)] = ten_by_ninety


def set_recent_stats_dow(df2, date, days, dow, mean, median, std, skew, kurtosis, ten_by_ninety):
    df2.loc[0,'date'] = date
    df2.loc[0, 'us_{}_dow{}_mean'.format(days, dow)] = mean
    df2.loc[0, 'us_{}_dow{}_med'.format(days, dow)] = median
    df2.loc[0, 'us_{}_dow{}_std'.format(days, dow)] = std
    df2.loc[0, 'us_{}_dow{}_skew'.format(days, dow)] = skew
    df2.loc[0, 'us_{}_dow{}_kurt'.format(days, dow)] = kurtosis
    df2.loc[0, 'us_{}_dow{}_tbn'.format(days, dow)] = ten_by_ninety

def set_recent_stats_promo_hol(df2, date, days, mean, median, std, skew, kurtosis, ten_by_ninety):
    df2.loc[0,'date'] = date
    df2.loc[0, 'us_{}_ph_mean'.format(days)] = mean
    df2.loc[0, 'us_{}_ph_med'.format(days)] = median
    df2.loc[0, 'us_{}_ph_std'.format(days)] = std
    df2.loc[0, 'us_{}_ph_skew'.format(days)] = skew
    df2.loc[0, 'us_{}_ph_kurt'.format(days)] = kurtosis
    df2.loc[0, 'us_{}_ph_tbn'.format(days)] = ten_by_ninety


def set_recent_stats_dow_promo_hol(df1, date, days, dow, mean, median, std, skew, kurtosis, ten_by_ninety):
    df2.loc[0,'date'] = date
    df2.loc[0, 'us_{}_dow{}_ph_mean'.format(days, dow)] = mean
    df2.loc[0, 'us_{}_dow{}_ph_med'.format(days, dow)] = median
    df2.loc[0, 'us_{}_dow{}_ph_std'.format(days, dow)] = std
    df2.loc[0, 'us_{}_dow{}_ph_skew'.format(days, dow)] = skew
    df2.loc[0, 'us_{}_dow{}_ph_kurt'.format(days, dow)] = kurtosis
    df2.loc[0, 'us_{}_dow{}_ph_tbn'.format(days, dow)] = ten_by_ninety


In [12]:
# df_h = pd.read_csv('../data/holidays_events.csv')

In [13]:
# df_h.type.value_counts()

Holiday       221
Event          56
Additional     51
Transfer       12
Work Day        5
Bridge          5
Name: type, dtype: int64

In [14]:
# df_h1 = df_h[(df.date.apply(np.datetime64)) > np.datetime64('2017-08-15')]

/home/watts/anaconda3/envs/rss/lib/python3.5/site-packages/ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.


In [15]:
# df.onpromotion.value_counts()

False    1574388
True      127620
Name: onpromotion, dtype: int64

In [16]:
# df.onpromotion[df.onpromotion==True].value_counts()

True    127620
Name: onpromotion, dtype: int64

In [17]:
# df.h_type.value_counts()

Work Day      1166317
Weekend        400151
Event           65901
Holiday         65721
Additional       3918
Name: h_type, dtype: int64

In [18]:
# df.h_type[df.h_type=='Work Day'].value_counts()

Work Day    1166317
Name: h_type, dtype: int64

In [19]:
# df.h_type[df.h_type != 'Work Day'].value_counts()

Weekend       400151
Event          65901
Holiday        65721
Additional      3918
Name: h_type, dtype: int64

In [20]:
#df_h.h_type[df_h.type == 'Additional'] = 'Holiday'

In [4]:
recent_days = [30, 90, 180, 365, 730]

In [5]:
overall_min_date = np.datetime64('2013-01-01')

In [6]:
my_min_date = np.datetime64('2015-01-01')

In [7]:
my_min_date - overall_min_date

numpy.timedelta64(730,'D')

In [8]:
def calculate_recent_stats(date, df, fn):
    df3 = pd.DataFrame()
    for days in recent_days:
        past_date = (np.datetime64(date) - np.timedelta64(days, 'D'))
        i = 0
        df_min_date = np.datetime64(np.min(df.date))
        while (past_date < df_min_date) & (i <= 0):
            past_date = (np.datetime64(date) - np.timedelta64(days-i, 'D'))
            i +=1
#         print(date, days, past_date, df_min_date)
        if past_date < df_min_date:
            print('date:{}, past_date:{}'.format(date, past_date))
            j = days-i
            print('days:{}, i: {}, days-i:{}'.format(days, i, j))
            set_recent_stats(df3, date,days,0, 0, 0, 0, 0, 0)
            continue
#         data = df.unit_sales[(df.dts < np.datetime64(date)) & 
#                              (df.dts >= past_date)]
        
        data = df.unit_sales[(df.date.astype('str') < str(date)) & 
                             (df.date.astype('str') >= str(past_date))]
        if(data.shape[0] == 0):
            print('date:{}, past_date:{}'.format(date, past_date))
            j = days-i
            print('days:{}, i: {}, days-i:{}'.format(days, i, j))
            set_recent_stats(df3, date,days,0, 0, 0, 0, 0, 0)
            continue
        mean = data.mean()
        median = data.median()
        std = data.std()
        skew = data.skew()
        kurtosis = data.kurtosis()
        tenp = data.quantile(0.1)
        ninetyp = data.quantile(0.9)
        ten_by_ninety = tenp/ninetyp
        set_recent_stats(df3, date,days,mean, median, std, skew, kurtosis, ten_by_ninety)
        del data
    append_to_csv(df3, fn)
    del df3
        

def calculate_recent_stats_promo(date, df, fn):
    df3 = pd.DataFrame()
    for days in recent_days:
        past_date = (np.datetime64(date) - np.timedelta64(days, 'D'))
        i = 0
        df_min_date = np.datetime64(np.min(df.date))
        while (past_date < df_min_date) & (i <= 0):
            past_date = (np.datetime64(date) - np.timedelta64(days-i, 'D'))
            i +=1
        
        if past_date < df_min_date:
            print('date:{}, past_date:{}'.format(date, past_date))
            j = days-i
            print('days:{}, i: {}, days-i:{}'.format(days, i, j))
            set_recent_stats(df3, date,days,0, 0, 0, 0, 0, 0)
            continue
        
        data = df.unit_sales[(df.dts < np.datetime64(date)) & 
                             (df.dts >= past_date) & (df.onpromotion == True)]
        if(data.shape[0] == 0):
            print('date:{}, past_date:{}'.format(date, past_date))
            j = days-i
            print('days:{}, i: {}, days-i:{}'.format(days, i, j))
            set_recent_stats(df3, date,days,0, 0, 0, 0, 0, 0)
            continue
        
        mean = data.mean()
        median = data.median()
        std = data.std()
        skew = data.skew()
        kurtosis = data.kurtosis()
        tenp = data.quantile(0.1)
        ninetyp = data.quantile(0.9)
        ten_by_ninety = tenp/ninetyp
        set_recent_stats_promo3(df3, date,days,mean, median, std, skew, kurtosis, ten_by_ninety)
        del data
    append_to_csv(df3, fn)
    del df3
        
        
def calculate_recent_stats_hol(date, df, fn):
    df3 = pd.DataFrame()
    for days in recent_days:
        past_date = (np.datetime64(date) - np.timedelta64(days, 'D'))
        i = 0
        df_min_date = np.datetime64(np.min(df.date))
        while (past_date < df_min_date) & (i <= 0):
            past_date = (np.datetime64(date) - np.timedelta64(days-i, 'D'))
            i +=1
        
        if past_date < df_min_date:
            print('date:{}, past_date:{}'.format(date, past_date))
            j = days-i
            print('days:{}, i: {}, days-i:{}'.format(days, i, j))
            set_recent_stats(df3, date,days,0, 0, 0, 0, 0, 0)
            continue
        
        data = df.unit_sales[(df.dts < np.datetime64(date)) & 
                             (df.dts >= past_date) & (df.h_type != 'Work Day')]
        if(data.shape[0] == 0):
            print('date:{}, past_date:{}'.format(date, past_date))
            j = days-i
            print('days:{}, i: {}, days-i:{}'.format(days, i, j))
            set_recent_stats(df3, date,days,0, 0, 0, 0, 0, 0)
            continue
        
        mean = data.mean()
        median = data.median()
        std = data.std()
        skew = data.skew()
        kurtosis = data.kurtosis()
        tenp = data.quantile(0.1)
        ninetyp = data.quantile(0.9)
        ten_by_ninety = tenp/ninetyp
        set_recent_stats_hol(df3, date,days,mean, median, std, skew, kurtosis, ten_by_ninety)
        del data
    append_to_csv(df3, fn)
    del df3
    
 
def calculate_recent_stats_perishable(date, df, fn):
    df3 = pd.DataFrame()
    for days in recent_days:
        past_date = (np.datetime64(date) - np.timedelta64(days, 'D'))
        i = 0
        df_min_date = np.datetime64(np.min(df.date))
        while (past_date < df_min_date) & (i <= 0):
            past_date = (np.datetime64(date) - np.timedelta64(days-i, 'D'))
            i +=1
        
        if past_date < df_min_date:
            print('date:{}, past_date:{}'.format(date, past_date))
            j = days-i
            print('days:{}, i: {}, days-i:{}'.format(days, i, j))
            set_recent_stats(df3, date,days,0, 0, 0, 0, 0, 0)
            continue
        
        data = df.unit_sales[(df.dts < np.datetime64(date)) & 
                             (df.dts >= past_date) & (df.perishable == 1)]
        if(data.shape[0] == 0):
            print('date:{}, past_date:{}'.format(date, past_date))
            j = days-i
            print('days:{}, i: {}, days-i:{}'.format(days, i, j))
            set_recent_stats(df3, date,days,0, 0, 0, 0, 0, 0)
            continue
        
        mean = data.mean()
        median = data.median()
        std = data.std()
        skew = data.skew()
        kurtosis = data.kurtosis()
        tenp = data.quantile(0.1)
        ninetyp = data.quantile(0.9)
        ten_by_ninety = tenp/ninetyp
        set_recent_stats_per(df3, date,days, mean, median, std, skew, kurtosis, ten_by_ninety)
        del data
    append_to_csv(df3, fn)
    del df3
        
def calculate_recent_stats_not_perishable(date, df, fn):
    df3 = pd.DataFrame()
    for days in recent_days:
        past_date = (np.datetime64(date) - np.timedelta64(days, 'D'))
        i = 0
        df_min_date = np.datetime64(np.min(df.date))
        while (past_date < df_min_date) & (i <= 0):
            past_date = (np.datetime64(date) - np.timedelta64(days-i, 'D'))
            i +=1
        
        if past_date < df_min_date:
            print('date:{}, past_date:{}'.format(date, past_date))
            j = days-i
            print('days:{}, i: {}, days-i:{}'.format(days, i, j))
            set_recent_stats(df3, date,days,0, 0, 0, 0, 0, 0)
            continue
        
        data = df.unit_sales[(df.dts < np.datetime64(date)) & 
                             (df.dts >= past_date) & (df.perishable == 0)]
        if(data.shape[0] == 0):
            print('date:{}, past_date:{}'.format(date, past_date))
            j = days-i
            print('days:{}, i: {}, days-i:{}'.format(days, i, j))
            set_recent_stats(df3, date,days,0, 0, 0, 0, 0, 0)
            continue
        mean = data.mean()
        median = data.median()
        std = data.std()
        skew = data.skew()
        kurtosis = data.kurtosis()
        tenp = data.quantile(0.1)
        ninetyp = data.quantile(0.9)
        ten_by_ninety = tenp/ninetyp
        set_recent_stats_nper(df3, date,days,mean, median, std, skew, kurtosis, ten_by_ninety)
        del data
    append_to_csv(df3, fn)
    del df3
        
def calculate_recent_stats_dow(date, dow, df, fn):
    df3 = pd.DataFrame()
    for days in recent_days:
        past_date = (np.datetime64(date) - np.timedelta64(days, 'D'))
        i = 0
        df_min_date = np.datetime64(np.min(df.date))
        while (past_date < df_min_date) & (i <= 0):
            past_date = (np.datetime64(date) - np.timedelta64(days-i, 'D'))
            i +=1
        
        if past_date < df_min_date:
            print('date:{}, past_date:{}'.format(date, past_date))
            j = days-i
            print('days:{}, i: {}, days-i:{}'.format(days, i, j))
            set_recent_stats(df3, date,days,0, 0, 0, 0, 0, 0)
            continue
        
        data = df.unit_sales[(df.dts < np.datetime64(date)) & 
                             (df.dts >= past_date) & (df.dow == dow)]
        if(data.shape[0] == 0):
            print('date:{}, past_date:{}'.format(date, past_date))
            j = days-i
            print('days:{}, i: {}, days-i:{}'.format(days, i, j))
            set_recent_stats(df3, date,days,0, 0, 0, 0, 0, 0)
            continue
        mean = data.mean()
        median = data.median()
        std = data.std()
        skew = data.skew()
        kurtosis = data.kurtosis()
        tenp = data.quantile(0.1)
        ninetyp = data.quantile(0.9)
        ten_by_ninety = tenp/ninetyp
        set_recent_stats_dow(df3, date,days,mean, median, std, skew, kurtosis, ten_by_ninety)
        del data
    append_to_csv(df3, fn)
    del df3
    
def calculate_recent_stats_promo_hol(date, df, fn):
    df3 = pd.DataFrame()
    for days in recent_days:
        past_date = (np.datetime64(date) - np.timedelta64(days, 'D'))
        i = 0
        df_min_date = np.datetime64(np.min(df.date))
        while (past_date < df_min_date) & (i <= 0):
            past_date = (np.datetime64(date) - np.timedelta64(days-i, 'D'))
            i +=1
        
        if past_date < df_min_date:
            print('date:{}, past_date:{}'.format(date, past_date))
            j = days-i
            print('days:{}, i: {}, days-i:{}'.format(days, i, j))
            set_recent_stats(df3, date,days,0, 0, 0, 0, 0, 0)
            continue
        
        data = df.unit_sales[(df.dts < np.datetime64(date)) & 
                             (df.dts >= past_date) & 
                             (df.h_type != 'Work Day') & (df.onpromotion == True)]
        if(data.shape[0] == 0):
            print('date:{}, past_date:{}'.format(date, past_date))
            j = days-i
            print('days:{}, i: {}, days-i:{}'.format(days, i, j))
            set_recent_stats(df3, date,days,0, 0, 0, 0, 0, 0)
            continue
        mean = data.mean()
        median = data.median()
        std = data.std()
        skew = data.skew()
        kurtosis = data.kurtosis()
        tenp = data.quantile(0.1)
        ninetyp = data.quantile(0.9)
        ten_by_ninety = tenp/ninetyp
        set_recent_stats_promo_hol(df3, date,days,mean, median, std, skew, kurtosis, ten_by_ninety)
        del data
    append_to_csv(df3, fn)
    del df3
    
def calculate_recent_stats_dow_promo_hol(date, dow, df, fn):
    df3 = pd.DataFrame()
    for days in recent_days:
        past_date = (np.datetime64(date) - np.timedelta64(days, 'D'))
        i = 0
        df_min_date = np.datetime64(np.min(df.date))
        while (past_date < df_min_date) & (i <= 0):
            past_date = (np.datetime64(date) - np.timedelta64(days-i, 'D'))
            i +=1
        
        if past_date < df_min_date:
            print('date:{}, past_date:{}'.format(date, past_date))
            j = days-i
            print('days:{}, i: {}, days-i:{}'.format(days, i, j))
            set_recent_stats(df3, date,days,0, 0, 0, 0, 0, 0)
            continue
        
        data = df.unit_sales[(df.dts < np.datetime64(date)) & 
                             (df.dts >= past_date) & 
                             (df.h_type != 'Work Day') & 
                             (df.onpromotion == True) & 
                             (df.dow == dow)]
        if(data.shape[0] == 0):
            print('date:{}, past_date:{}'.format(date, past_date))
            j = days-i
            print('days:{}, i: {}, days-i:{}'.format(days, i, j))
            set_recent_stats(df3, date,days,dow,0, 0, 0, 0, 0, 0)
            continue
        mean = data.mean()
        median = data.median()
        std = data.std()
        skew = data.skew()
        kurtosis = data.kurtosis()
        tenp = data.quantile(0.1)
        ninetyp = data.quantile(0.9)
        ten_by_ninety = tenp/ninetyp
        set_recent_stats_dow_promo_hol(df3, date,days,dow,mean, median, std, skew, kurtosis, ten_by_ninety)
        del data
    append_to_csv(df3, fn)
    del df3

In [32]:
print(df.unit_sales.mean())

2.02009826006


In [35]:
past_date = (np.datetime64('2017-08-21') - np.timedelta64(2, 'D'))

In [42]:
print(past_date)

2017-08-19


In [40]:
df['dts'] = df.date.map(np.datetime64)

In [52]:
df.loc[0, 'date']

'2017-08-16'

In [53]:
df[(df.date.astype('str') >= str(past_date))]

,id,date,store_nbr,item_nbr,onpromotion,item_family,item_class,perishable,city,state,...,dcount,h_type,h_desc,pd,wbe,wae,wfe,store_item_nbr,unit_sales,dts
11703,126129002,2017-08-19,1,96995,False,GROCERY I,1093,0,Quito,Pichincha,...,736560,Weekend,Normal,0,0,70,70,1_96995,0.2795,2017-08-19
11704,126129003,2017-08-19,1,99197,False,GROCERY I,1067,0,Quito,Pichincha,...,736560,Weekend,Normal,0,0,70,70,1_99197,0.5001,2017-08-19
11705,126129004,2017-08-19,1,103501,False,CLEANING,3008,0,Quito,Pichincha,...,736560,Weekend,Normal,0,0,70,70,1_103501,0.0000,2017-08-19
11706,126129005,2017-08-19,1,103520,False,GROCERY I,1028,0,Quito,Pichincha,...,736560,Weekend,Normal,0,0,70,70,1_103520,1.1034,2017-08-19
11707,126129006,2017-08-19,1,103665,False,BREAD/BAKERY,2712,1,Quito,Pichincha,...,736560,Weekend,Normal,0,0,70,70,1_103665,2.4947,2017-08-19
11708,126129007,2017-08-19,1,105574,False,GROCERY I,1045,0,Quito,Pichincha,...,736560,Weekend,Normal,0,0,70,70,1_105574,4.3018,2017-08-19
11709,126129008,2017-08-19,1,105575,False,GROCERY I,1045,0,Quito,Pichincha,...,736560,Weekend,Normal,0,0,70,70,1_105575,9.7603,2017-08-19
11710,126129009,2017-08-19,1,105576,False,GROCERY I,1045,0,Quito,Pichincha,...,736560,Weekend,Normal,0,0,70,70,1_105576,0.0000,2017-08-19
11711,126129010,2017-08-19,1,105577,False,GROCERY I,1045,0,Quito,Pichincha,...,736560,Weekend,Normal,0,0,70,70,1_105577,0.6093,2017-08-19
11712,126129011,2017-08-19,1,105693,False,GROCERY I,1034,0,Quito,Pichincha,...,736560,Weekend,Normal,0,0,70,70,1_105693,0.4281,2017-08-19


In [34]:
df.unit_sales[(df.date.apply(np.datetime64) >= past_date) & 
              (df.date.apply(np.datetime64) < np.datetime64('2016-01-01'))].mean()

ValueError: cannot set a Timestamp with a non-timestamp

In [29]:
df1 = df.set_index('date')

In [30]:
s1 = df1.loc['2015-12-31', 'unit_sales']
s2 = df1.loc['2015-12-30', 'unit_sales']

In [31]:
s = pd.concat([s1, s2], axis=0)

In [32]:
s.mean()

6.5914254756871031

In [33]:
gc.enable()

In [34]:
# import os
# x = {i for i in range(20)}
# os.sched_setaffinity(0, x)

In [35]:
# chunks = []
# for i in range(1, len(d), 1000):
#    chunks.append(d[i:min(i + 1000, len(d))])

# res = Parallel(n_jobs=-1) (delayed(hash) (d) for d in chunks)

# def hash(d):
#     return [hashlib.sha256(str(x).encode('utf-8')).hexdigest()[:30] for x in d.itertuples(index=False, name=None)]

In [36]:
# Parallel(n_jobs=20)(delayed(sqrt)(i ** 2) for i in range(10000000))

In [37]:
# df = pd.DataFrame()
# df1 = pd.DataFrame()

In [38]:
# for i in range(1,55):
#     print(i)
#     fn = '../cache/train10_t_store{}.csv'.format(i)
#     df2 = pd.read_csv(fn)
#     df2.unit_sales[df2.unit_sales < 0] = 0
#     df3 = df2.set_index('date')
#     dates = np.unique(df2.date.values)
# #     for j, date in tqdm(enumerate(dates)):
#     df = df2.copy()
#     df1 = df3.copy()
#     df = df.iloc[::-1]
#     Parallel(n_jobs=10)(delayed(calculate_recent_stats)(date) for date in tqdm(dates))
# #         calculate_recent_stats(date, df, df1)
#     for days in [30, 90, 180, 365, 730]:
#         df = set_recent_df_stats(df ,df1, days)
#     df = df.iloc[::-1]
#     print(df.shape)
#     print(sum(df.isnull().any()==True)) # print NA
#     print(sum((df != 0).any(axis=0))) # print non-zero columns
#     fn = '../cache/train10_t_store{}_recent_stats.csv'.format(i)
#     print(fn)
#     df.to_csv(fn, index=False)
#     del df2
#     del df3

In [39]:
# files_list = []
# for i in range(1,55):
#     fn = '../cache/train10_t_store{}.csv'.format(i)
#     files_list.append(fn)

In [40]:
# print('creating pool..')
# print(cpu_count())
# thread_pool = Pool(10)
# print('mapping pool..')
# thread_pool.map (calculate_recent_stats2, arg_list)

# print('joining pool..')

In [9]:
# from multiprocessing import Pool, cpu_count
def process_csv_file2(i):
    fn = '../cache/bkup/train10_t_store{}.csv'.format(i)
    print(fn)
    df = pd.read_csv(fn)
    print(df.shape)
    df.unit_sales[df.unit_sales < 0] = 0 
    df = df.iloc[::-1]
    df1 = df.set_index('date')

    trend_dates = []
    for date in np.unique(df.date.values):
        curr = np.datetime64(date)
        last_in_trend = np.datetime64('2015-01-01')
        if curr < last_in_trend:
            continue
        trend_dates.append(date)
    for date in tqdm(trend_dates):
        calculate_recent_stats(date, df, df1)
    for days in [30, 90, 180, 365, 730]:
        df = set_recent_df_stats(df, df1, days)
    df = df.iloc[::-1]
    
    df = df[(df.date.apply(np.datetime64) >= np.datetime64('2015-01-01')) & 
              (df.date.apply(np.datetime64) <= np.datetime64('2017-08-15'))]
    
    print(df.shape)
    s = sum(df.isnull().any()==True) # print NA
    print(s)
    if s > 0:
        print(df.isnull().sum())
    print(sum((df != 0).any(axis=0))) # print non-zero columns
    fn = '../cache/train10_t_store{}_recent_stats.csv'.format(i)
    print(fn)
    df.to_csv(fn, index=False)
    del df
    del df1

In [10]:
def delete_file_if_exists(filename):
    if os.path.exists(filename):
        os.remove(filename)

In [11]:
def process_csv_file1(i):
    fn_org = '../cache/bkup/train10_t_store{}.csv'.format(i)
    print(fn_org)
    df = pd.read_csv(fn_org)
    
    df.unit_sales[df.unit_sales < 0] = 0 
    df['dts'] = df.date.map(np.datetime64)

    trend_dates = []
    for date in np.unique(df.date.values):
        curr = np.datetime64(date)
        last_in_trend = np.datetime64('2015-01-01')
        if curr < last_in_trend:
            continue
        trend_dates.append(date)
    
    fn_tmp = '../cache/train10_t_store{}_recent_stats_tmp.csv'.format(i)
    delete_file_if_exists(fn_tmp)
    for date in tqdm(trend_dates):
        calculate_recent_stats(date, df, fn_tmp)

    print(df.shape)
    df = df[(df.dts >= np.datetime64('2015-01-01')) & (df.dts <= np.datetime64('2017-08-15'))]

    print(df.shape)
    nrows1 = df.shape[0]
    df1 = pd.read_csv(fn_tmp)
    print(df1.shape)
    df = pd.merge(df, df1, how='left', on='date')
    df = df.drop('dts', axis=1)
    s = sum(df.isnull().any()==True) # print NA
    print(s)
    if s > 0:
        print(df.isnull().sum())
    print(sum((df != 0).any(axis=0))) # print non-zero columns
    fn = '../cache/train10_t_store{}_recent_stats.csv'.format(i)
    print(fn)
    df.to_csv(fn, index=False)
    print(df.shape)
    nrows2 = df.shape[0]
    assert(nrows1 == nrows2)
    del df
    del df1

In [12]:
def process_csv_file(i):
    fn_org = '../cache/bkup/train10_t_store{}.csv'.format(i)
    print(fn_org)
    df = pd.read_csv(fn_org)
    
#     df.unit_sales[df.unit_sales < 0] = 0 
    df['dts'] = df.date.map(np.datetime64)

    trend_dates = []
    for date in np.unique(df.date.values):
        curr = np.datetime64(date)
        last_in_trend = np.datetime64('2015-01-01')
        if curr < last_in_trend:
            continue
        trend_dates.append(date)
    
    fn_tmp = '../cache/train10_t_store{}_recent_stats_tmp.csv'.format(i)
    delete_file_if_exists(fn_tmp)
    for date in tqdm(trend_dates):
        calculate_recent_stats(date, df, fn_tmp)

    print(df.shape)
    df = df[(df.dts >= np.datetime64('2015-01-01')) & (df.dts <= np.datetime64('2017-08-15'))]

    print(df.shape)
    nrows1 = df.shape[0]
    df1 = pd.read_csv(fn_tmp)
    print(df1.shape)
    df = pd.merge(df, df1, how='left', on='date')
    df = df.drop('dts', axis=1)
    s = sum(df.isnull().any()==True) # print NA
    print(s)
    if s > 0:
        print(df.isnull().sum())
    print(sum((df != 0).any(axis=0))) # print non-zero columns
    fn = '../cache/train10_t_store{}_recent_stats_1.csv'.format(i)
    print(fn)
    df.to_csv(fn, float_format='%.5f', index=False)
    print(df.shape)
    nrows2 = df.shape[0]
    assert(nrows1 == nrows2)
    del df
    del df1
    delete_file_if_exists(fn_tmp)


In [9]:
def process_test_csv_file(i):
    fn_org = '../cache/bkup/train10_t_store{}.csv'.format(i)
    fn_org1 = '../cache/bkup2/test7_1_t_store{}.csv'.format(i)
    print(fn_org1)
    df = pd.read_csv(fn_org)
    df1 = pd.read_csv(fn_org1)
    df = pd.concat([df, df1], axis=0)
    
#     df.unit_sales[df.unit_sales < 0] = 0 
    df['dts'] = df.date.map(np.datetime64)

    trend_dates = []
    for date in np.unique(df.date.values):
        curr = np.datetime64(date)
        last_in_trend = np.datetime64('2017-08-16')
        if curr < last_in_trend:
            continue
        trend_dates.append(date)
    
    fn_tmp = '../cache/test7_1_t_store{}_recent_stats_tmp.csv'.format(i)
    delete_file_if_exists(fn_tmp)
    for date in tqdm(trend_dates):
        calculate_recent_stats(date, df, fn_tmp)

    print(df.shape)
#     df = df[(df.dts >= np.datetime64('2017-08-16')) & (df.dts <= np.datetime64('2017-08-31'))]
    
    df = df[(df.date.astype('str') >= str('2017-08-16')) & (df.date.astype('str') <= str('2017-08-31'))]

    print(df.shape)
    nrows1 = df.shape[0]
    df1 = pd.read_csv(fn_tmp)
    print(df1.shape)
    df = pd.merge(df, df1, how='left', on='date')
    df = df.drop('dts', axis=1)
    s = sum(df.isnull().any()==True) # print NA
    print(s)
    if s > 0:
        print(df.isnull().sum())
    print(sum((df != 0).any(axis=0))) # print non-zero columns
    fn = '../cache/test7_1_t_store{}_recent_stats_1.csv'.format(i)
    print(fn)
    df.to_csv(fn, float_format='%.5f', index=False)
    print(df.shape)
    nrows2 = df.shape[0]
    assert(nrows1 == nrows2)
    del df
    del df1
    delete_file_if_exists(fn_tmp)


In [9]:
date_block = {}
date_block['1_2013'] = 0
date_block['2_2013'] = 1
date_block['3_2013'] = 2
date_block['4_2013'] = 3
date_block['5_2013'] = 4
date_block['6_2013'] = 5
date_block['7_2013'] = 6
date_block['8_2013'] = 7
date_block['9_2013'] = 8
date_block['10_2013'] = 9
date_block['11_2013'] = 10
date_block['12_2013'] = 11

date_block['1_2014'] = 12
date_block['2_2014'] = 13
date_block['3_2014'] = 14
date_block['4_2014'] = 15
date_block['5_2014'] = 16
date_block['6_2014'] = 17
date_block['7_2014'] = 18
date_block['8_2014'] = 19
date_block['9_2014'] = 20
date_block['10_2014'] = 21
date_block['11_2014'] = 22
date_block['12_2014'] = 23

date_block['1_2015'] = 24
date_block['2_2015'] = 25
date_block['3_2015'] = 26
date_block['4_2015'] = 27
date_block['5_2015'] = 28
date_block['6_2015'] = 29
date_block['7_2015'] = 30
date_block['8_2015'] = 31
date_block['9_2015'] = 32
date_block['10_2015'] = 33
date_block['11_2015'] = 34
date_block['12_2015'] = 35

date_block['1_2016'] = 36
date_block['2_2016'] = 37
date_block['3_2016'] = 38
date_block['4_2016'] = 39
date_block['5_2016'] = 40
date_block['6_2016'] = 41
date_block['7_2016'] = 42
date_block['8_2016'] = 43
date_block['9_2016'] = 44
date_block['10_2016'] = 45
date_block['11_2016'] = 46
date_block['12_2016'] = 47

date_block['1_2017'] = 48
date_block['2_2017'] = 49
date_block['3_2017'] = 50
date_block['4_2017'] = 51
date_block['5_2017'] = 52
date_block['6_2017'] = 53
date_block['7_2017'] = 54
date_block['8_2017'] = 55
date_block['9_2017'] = 56
date_block['10_2017'] = 57
date_block['11_2017'] = 58
date_block['12_2017'] = 59


In [10]:
def get_date_block_num(x):
    year, month, day = x.split('-')
    mon = int(month)
    yr = int(year)
    mon_year = str(mon) + '_' + str(yr)
    
    return date_block[mon_year]

In [ ]:
for shop_num in tqdm(range(1,55)):
    fn_org = '../cache/bkup/train10_t_store{}.csv'.format(shop_num)
    fn_org1 = '../cache/bkup2/test7_1_t_store{}.csv'.format(shop_num)
    print(fn_org1)
    df = pd.read_csv(fn_org)
    df1 = pd.read_csv(fn_org1)
    df = pd.concat([df, df1], axis=0)
    
    print(df.shape)
    df['date_block_num']  = df.date.map(lambda x: get_date_block_num(x))
#     df = df.drop_duplicates()
    unit_sales_mean = df.groupby(['item_nbr', 'date_block_num'], as_index=False).agg({'unit_sales':{'unit_sales_mean':'mean'}, 
                                                         })
    unit_sales_mean.columns = ['item_nbr', 'date_block_num', 'unit_sales_mean']
    df = pd.merge(df, unit_sales_mean, how='left', on=['item_nbr', 'date_block_num']).fillna(0.0)
    
    shifted_df = df[['id', 'date', 'date_block_num','item_nbr','unit_sales', 'unit_sales_mean', 'unit_sales_median', 'unit_sales_std']].copy()
    
    fn = '../cache/shifted_rs_df_shop{}.csv'.format(shop_num)
    delete_file_if_exists(fn)
    
    shifted_df.to_csv(fn, float_format='%.5f', index=False)
    del shifted_df
    del df1
    del df
    del unit_sales_mean

In [33]:
for shop_num in tqdm(range(1,55)):
    fn_org = '../cache/bkup/train10_t_store{}.csv'.format(shop_num)
    fn_org1 = '../cache/bkup2/test7_1_t_store{}.csv'.format(shop_num)
    print(fn_org1)
    df = pd.read_csv(fn_org)
    df1 = pd.read_csv(fn_org1)
    df = pd.concat([df, df1], axis=0)
    
    print(df.shape)
    df['date_block_num']  = df.date.map(lambda x: get_date_block_num(x))
#     df = df.drop_duplicates()
    unit_sales_mean = df.groupby(['item_nbr', 'date_block_num'], as_index=False).agg({'unit_sales':{'unit_sales_mean':'mean'}, 
                                                         })
    unit_sales_mean.columns = ['item_nbr', 'date_block_num', 'unit_sales_mean']
    df = pd.merge(df, unit_sales_mean, how='left', on=['item_nbr', 'date_block_num']).fillna(0.0)

    unit_sales_median = df.groupby(['item_nbr', 'date_block_num'], as_index=False).agg({'unit_sales':{'unit_sales_median':'median'}, 
                                                         })
    unit_sales_median.columns = ['item_nbr', 'date_block_num', 'unit_sales_median']
    df = pd.merge(df, unit_sales_median, how='left', on=['item_nbr', 'date_block_num']).fillna(0.0)
    
    unit_sales_std = df.groupby(['item_nbr', 'date_block_num'], as_index=False).agg({'unit_sales':{'unit_sales_std':'std'}, 
                                                         })
    unit_sales_std.columns = ['item_nbr', 'date_block_num', 'unit_sales_std']
    df = pd.merge(df, unit_sales_std, how='left', on=['item_nbr', 'date_block_num']).fillna(0.0)
    
    
    shifted_df = df[['id', 'date', 'date_block_num','item_nbr','unit_sales_mean', 'unit_sales_median', 'unit_sales_std']].copy()
    
    fn = '../cache/shifted_rs_df_shop1{}.csv'.format(shop_num)
    delete_file_if_exists(fn)
    
    shifted_df.to_csv(fn, float_format='%.5f', index=False)
    del shifted_df
    del df1
    del df
    del unit_sales_mean
    del unit_sales_median
    del unit_sales_std


  0%|          | 0/54 [00:00<?, ?it/s]

../cache/bkup2/test7_1_t_store1.csv


/home/watts/anaconda2/envs/sdp35/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


(2624569, 27)



Exception in thread Thread-19:
Traceback (most recent call last):
  File "/home/watts/anaconda2/envs/sdp35/lib/python3.5/threading.py", line 914, in _bootstrap_inner
    self.run()
  File "/home/watts/anaconda2/envs/sdp35/lib/python3.5/site-packages/tqdm/_tqdm.py", line 144, in run
    for instance in self.tqdm_cls._instances:
  File "/home/watts/anaconda2/envs/sdp35/lib/python3.5/_weakrefset.py", line 60, in __iter__
    for itemref in self.data:
RuntimeError: Set changed size during iteration

/home/watts/anaconda2/envs/sdp35/lib/python3.5/site-packages/pandas/core/groupby.py:4189: FutureWarning: using a dict with renaming is deprecated and will be removed in a future version
  return super(DataFrameGroupBy, self).aggregate(arg, *args, **kwargs)
  2%|▏         | 1/54 [00:36<31:48, 36.00s/it]

...

 96%|█████████▋| 52/54 [28:49<01:06, 33.26s/it]

../cache/bkup2/test7_1_t_store53.csv
(2000671, 27)


 98%|█████████▊| 53/54 [29:16<00:33, 33.14s/it]

../cache/bkup2/test7_1_t_store54.csv
(1711283, 27)


100%|██████████| 54/54 [29:39<00:00, 32.96s/it]


In [26]:
# from tqdm import tqdm
for shop_num in tqdm(range(1,55)):
    fn = '../cache/shifted_rs_df_shop{}.csv'.format(shop_num)
    df1 = pd.read_csv(fn)

#     print(df1.shape)
#     print(set(df1.date_block_num))
#     df1 = df1.drop('my_id', axis=1)
    # forward
    # 1 month
    shifted_df = df1[['date_block_num','item_nbr','unit_sales_mean']].copy()
#     print(shifted_df.shape)
    org_date_blocks = shifted_df.date_block_num.values
    shifted_df.date_block_num += 1
    shifted_df.columns = ['date_block_num','item_nbr', 'unit_sales_mean_f_lag_1']
#     print(shifted_df.date_block_num.nunique())
    
    df1 = df1.merge(shifted_df, on=['date_block_num','item_nbr'], how='left').fillna(0.0)
    df1 = df1.drop_duplicates()
    
    assert(len(df1) == len(shifted_df))
    
    df = pd.DataFrame()
    df['id'] = df1.id.values
    df['unit_sales_mean_f_lag_1'] = df1.unit_sales_mean_f_lag_1.values
    df = df.loc[:, (df != 0).any(axis=0)]
    if len(df.columns) != 2:
        print(shop_num)
        print(df1.shape)
        print(set(df1.date_block_num))
        print(shifted_df.date_block_num.nunique())
        print(set(shifted_df.date_block_num))
        print(df.shape)
        print('------')
    fn = '../cache/lag1_sales_mean_rs_df_shop{}.csv'.format(shop_num)
    delete_file_if_exists(fn)
    df.to_csv(fn, float_format='%.5f', index=False)
    del shifted_df
    del df1
    del df
    
    

100%|██████████| 54/54 [19:07<00:00, 21.26s/it]


In [27]:
# for shop_num in tqdm(range(1,55)):
#     fn = '../cache/lag1_sales_mean_df_shop{}.csv'.format(shop_num)
#     df1 = pd.read_csv(fn)
#     fn1 = '../cache/lag1_sales_mean_rs_df_shop{}.csv'.format(shop_num)
#     df1.to_csv(fn1, float_format='%.5f', index=False)
#     delete_file_if_exists(fn)
#     del df1

In [28]:
# from tqdm import tqdm
for shop_num in tqdm(range(1,55)):
    fn = '../cache/shifted_rs_df_shop{}.csv'.format(shop_num)
    df1 = pd.read_csv(fn)

#     print(df1.shape)
#     print(set(df1.date_block_num))
#     df1 = df1.drop('my_id', axis=1)
    # forward
    # 1 month
    shifted_df = df1[['date_block_num','item_nbr','unit_sales_mean']].copy()
#     print(shifted_df.shape)
    org_date_blocks = shifted_df.date_block_num.values
    shifted_df.date_block_num += 3
    shifted_df.columns = ['date_block_num','item_nbr', 'unit_sales_mean_f_lag_3']
#     print(shifted_df.date_block_num.nunique())
    
    df1 = df1.merge(shifted_df, on=['date_block_num','item_nbr'], how='left').fillna(0.0)
    df1 = df1.drop_duplicates()
    
    assert(len(df1) == len(shifted_df))
    
    df = pd.DataFrame()
    df['id'] = df1.id.values
    df['unit_sales_mean_f_lag_3'] = df1.unit_sales_mean_f_lag_3.values
    df = df.loc[:, (df != 0).any(axis=0)]
    if len(df.columns) != 2:
        print(shop_num)
        print(df1.shape)
        print(set(df1.date_block_num))
        print(shifted_df.date_block_num.nunique())
        print(set(shifted_df.date_block_num))
        print(df.shape)
        print('------')
    fn = '../cache/lag3_sales_mean_rs_df_shop{}.csv'.format(shop_num)
    delete_file_if_exists(fn)
    df.to_csv(fn, float_format='%.5f', index=False)
    del shifted_df
    del df1
    del df
    
    

100%|██████████| 54/54 [18:17<00:00, 20.33s/it]


In [29]:
# from tqdm import tqdm
for shop_num in tqdm(range(1,55)):
    fn = '../cache/shifted_rs_df_shop{}.csv'.format(shop_num)
    df1 = pd.read_csv(fn)

#     print(df1.shape)
#     print(set(df1.date_block_num))
#     df1 = df1.drop('my_id', axis=1)
    # forward
    # 1 month
    shifted_df = df1[['date_block_num','item_nbr','unit_sales_mean']].copy()
#     print(shifted_df.shape)
    org_date_blocks = shifted_df.date_block_num.values
    shifted_df.date_block_num += 6
    shifted_df.columns = ['date_block_num','item_nbr', 'unit_sales_mean_f_lag_6']
#     print(shifted_df.date_block_num.nunique())
    
    df1 = df1.merge(shifted_df, on=['date_block_num','item_nbr'], how='left').fillna(0.0)
    df1 = df1.drop_duplicates()
    
    assert(len(df1) == len(shifted_df))
    
    df = pd.DataFrame()
    df['id'] = df1.id.values
    df['unit_sales_mean_f_lag_6'] = df1.unit_sales_mean_f_lag_6.values
    df = df.loc[:, (df != 0).any(axis=0)]
    if len(df.columns) != 2:
        print(shop_num)
        print(df1.shape)
        print(set(df1.date_block_num))
        print(shifted_df.date_block_num.nunique())
        print(set(shifted_df.date_block_num))
        print(df.shape)
        print('------')
    fn = '../cache/lag6_sales_mean_rs_df_shop{}.csv'.format(shop_num)
    delete_file_if_exists(fn)
    df.to_csv(fn, float_format='%.5f', index=False)
    del shifted_df
    del df1
    del df
    

 94%|█████████▍| 51/54 [16:52<00:59, 19.85s/it]

52
(352997, 7)
{51, 52, 53, 54, 55}
5
{57, 58, 59, 60, 61}
(352997, 1)
------


100%|██████████| 54/54 [17:21<00:00, 19.28s/it]


In [30]:
# from tqdm import tqdm
for shop_num in tqdm(range(1,55)):
    fn = '../cache/shifted_rs_df_shop{}.csv'.format(shop_num)
    df1 = pd.read_csv(fn)

#     print(df1.shape)
#     print(set(df1.date_block_num))
#     df1 = df1.drop('my_id', axis=1)
    # forward
    # 1 month
    shifted_df = df1[['date_block_num','item_nbr','unit_sales_mean']].copy()
#     print(shifted_df.shape)
    org_date_blocks = shifted_df.date_block_num.values
    shifted_df.date_block_num += 12
    shifted_df.columns = ['date_block_num','item_nbr', 'unit_sales_mean_f_lag_12']
#     print(shifted_df.date_block_num.nunique())
    
    df1 = df1.merge(shifted_df, on=['date_block_num','item_nbr'], how='left').fillna(0.0)
    df1 = df1.drop_duplicates()
    
    assert(len(df1) == len(shifted_df))
    
    df = pd.DataFrame()
    df['id'] = df1.id.values
    df['unit_sales_mean_f_lag_12'] = df1.unit_sales_mean_f_lag_12.values
    df = df.loc[:, (df != 0).any(axis=0)]
    if len(df.columns) != 2:
        print(shop_num)
        print(df1.shape)
        print(set(df1.date_block_num))
        print(shifted_df.date_block_num.nunique())
        print(set(shifted_df.date_block_num))
        print(df.shape)
        print('------')
    fn = '../cache/lag12_sales_mean_rs_df_shop{}.csv'.format(shop_num)
    delete_file_if_exists(fn)
    df.to_csv(fn, float_format='%.5f', index=False)
    del shifted_df
    del df1
    del df

 94%|█████████▍| 51/54 [15:18<00:54, 18.02s/it]

52
(352997, 7)
{51, 52, 53, 54, 55}
5
{64, 65, 66, 67, 63}
(352997, 1)
------


100%|██████████| 54/54 [15:44<00:00, 17.50s/it]


In [31]:
# from tqdm import tqdm
for shop_num in tqdm(range(1,55)):
    fn = '../cache/shifted_rs_df_shop{}.csv'.format(shop_num)
    df1 = pd.read_csv(fn)

#     print(df1.shape)
#     print(set(df1.date_block_num))
#     df1 = df1.drop('my_id', axis=1)
    # forward
    # 1 month
    shifted_df = df1[['date_block_num','item_nbr','unit_sales_mean']].copy()
#     print(shifted_df.shape)
    org_date_blocks = shifted_df.date_block_num.values
    shifted_df.date_block_num += 24
    shifted_df.columns = ['date_block_num','item_nbr', 'unit_sales_mean_f_lag_24']
#     print(shifted_df.date_block_num.nunique())
    
    df1 = df1.merge(shifted_df, on=['date_block_num','item_nbr'], how='left').fillna(0.0)
    df1 = df1.drop_duplicates()
    
    assert(len(df1) == len(shifted_df))
    
    df = pd.DataFrame()
    df['id'] = df1.id.values
    df['unit_sales_mean_f_lag_24'] = df1.unit_sales_mean_f_lag_24.values
    df = df.loc[:, (df != 0).any(axis=0)]
    if len(df.columns) != 2:
        print(shop_num)
        print(df1.shape)
        print(set(df1.date_block_num))
        print(shifted_df.date_block_num.nunique())
        print(set(shifted_df.date_block_num))
        print(df.shape)
        print('------')
    fn = '../cache/lag24_sales_mean_rs_df_shop{}.csv'.format(shop_num)
    delete_file_if_exists(fn)
    df.to_csv(fn, float_format='%.5f', index=False)
    del shifted_df
    del df1
    del df

 39%|███▉      | 21/54 [04:59<07:49, 14.24s/it]

22
(985899, 7)
{33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55}
23
{57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79}
(985899, 1)
------


 94%|█████████▍| 51/54 [12:21<00:43, 14.54s/it]

52
(352997, 7)
{51, 52, 53, 54, 55}
5
{75, 76, 77, 78, 79}
(352997, 1)
------


100%|██████████| 54/54 [12:42<00:00, 14.12s/it]


In [35]:
for shop_num in tqdm(range(1,55)):
    fn = '../cache/shifted_rs_df_shop1{}.csv'.format(shop_num)
    df1 = pd.read_csv(fn)

#     print(df1.shape)
#     print(set(df1.date_block_num))
#     df1 = df1.drop('my_id', axis=1)
    # forward
    # 1 month
    shifted_df = df1[['date_block_num','item_nbr','unit_sales_median']].copy()
#     print(shifted_df.shape)
    org_date_blocks = shifted_df.date_block_num.values
    shifted_df.date_block_num += 1
    shifted_df.columns = ['date_block_num','item_nbr', 'unit_sales_median_f_lag_1']
#     print(shifted_df.date_block_num.nunique())
    
    df1 = df1.merge(shifted_df, on=['date_block_num','item_nbr'], how='left').fillna(0.0)
    df1 = df1.drop_duplicates()
    
    assert(len(df1) == len(shifted_df))
    
    df = pd.DataFrame()
    df['id'] = df1.id.values
    df['unit_sales_median_f_lag_1'] = df1.unit_sales_median_f_lag_1.values
    df = df.loc[:, (df != 0).any(axis=0)]
    if len(df.columns) != 2:
        print(shop_num)
        print(df1.shape)
        print(set(df1.date_block_num))
        print(shifted_df.date_block_num.nunique())
        print(set(shifted_df.date_block_num))
        print(df.shape)
        print('------')
    fn = '../cache/lag1_sales_median_rs_df_shop{}.csv'.format(shop_num)
    delete_file_if_exists(fn)
    df.to_csv(fn, float_format='%.5f', index=False)
    del shifted_df
    del df1
    del df


  0%|          | 0/54 [00:00<?, ?it/s]
Exception in thread Thread-20:
Traceback (most recent call last):
  File "/home/watts/anaconda2/envs/sdp35/lib/python3.5/threading.py", line 914, in _bootstrap_inner
    self.run()
  File "/home/watts/anaconda2/envs/sdp35/lib/python3.5/site-packages/tqdm/_tqdm.py", line 144, in run
    for instance in self.tqdm_cls._instances:
  File "/home/watts/anaconda2/envs/sdp35/lib/python3.5/_weakrefset.py", line 60, in __iter__
    for itemref in self.data:
RuntimeError: Set changed size during iteration

100%|██████████| 54/54 [20:08<00:00, 22.38s/it]


In [36]:
for shop_num in tqdm(range(1,55)):
    fn = '../cache/shifted_rs_df_shop1{}.csv'.format(shop_num)
    df1 = pd.read_csv(fn)

#     print(df1.shape)
#     print(set(df1.date_block_num))
#     df1 = df1.drop('my_id', axis=1)
    # forward
    # 1 month
    shifted_df = df1[['date_block_num','item_nbr','unit_sales_median']].copy()
#     print(shifted_df.shape)
    org_date_blocks = shifted_df.date_block_num.values
    shifted_df.date_block_num += 3
    shifted_df.columns = ['date_block_num','item_nbr', 'unit_sales_median_f_lag_3']
#     print(shifted_df.date_block_num.nunique())
    
    df1 = df1.merge(shifted_df, on=['date_block_num','item_nbr'], how='left').fillna(0.0)
    df1 = df1.drop_duplicates()
    
    assert(len(df1) == len(shifted_df))
    
    df = pd.DataFrame()
    df['id'] = df1.id.values
    df['unit_sales_median_f_lag_3'] = df1.unit_sales_median_f_lag_3.values
    df = df.loc[:, (df != 0).any(axis=0)]
    if len(df.columns) != 2:
        print(shop_num)
        print(df1.shape)
        print(set(df1.date_block_num))
        print(shifted_df.date_block_num.nunique())
        print(set(shifted_df.date_block_num))
        print(df.shape)
        print('------')
    fn = '../cache/lag3_sales_median_rs_df_shop{}.csv'.format(shop_num)
    delete_file_if_exists(fn)
    df.to_csv(fn, float_format='%.5f', index=False)
    del shifted_df
    del df1
    del df

100%|██████████| 54/54 [19:13<00:00, 21.36s/it]


In [37]:
for shop_num in tqdm(range(1,55)):
    fn = '../cache/shifted_rs_df_shop1{}.csv'.format(shop_num)
    df1 = pd.read_csv(fn)

#     print(df1.shape)
#     print(set(df1.date_block_num))
#     df1 = df1.drop('my_id', axis=1)
    # forward
    # 1 month
    shifted_df = df1[['date_block_num','item_nbr','unit_sales_median']].copy()
#     print(shifted_df.shape)
    org_date_blocks = shifted_df.date_block_num.values
    shifted_df.date_block_num += 6
    shifted_df.columns = ['date_block_num','item_nbr', 'unit_sales_median_f_lag_6']
#     print(shifted_df.date_block_num.nunique())
    
    df1 = df1.merge(shifted_df, on=['date_block_num','item_nbr'], how='left').fillna(0.0)
    df1 = df1.drop_duplicates()
    
    assert(len(df1) == len(shifted_df))
    
    df = pd.DataFrame()
    df['id'] = df1.id.values
    df['unit_sales_median_f_lag_6'] = df1.unit_sales_median_f_lag_6.values
    df = df.loc[:, (df != 0).any(axis=0)]
    if len(df.columns) != 2:
        print(shop_num)
        print(df1.shape)
        print(set(df1.date_block_num))
        print(shifted_df.date_block_num.nunique())
        print(set(shifted_df.date_block_num))
        print(df.shape)
        print('------')
    fn = '../cache/lag6_sales_median_rs_df_shop{}.csv'.format(shop_num)
    delete_file_if_exists(fn)
    df.to_csv(fn, float_format='%.5f', index=False)
    del shifted_df
    del df1
    del df

 94%|█████████▍| 51/54 [17:41<01:02, 20.81s/it]

52
(352997, 8)
{51, 52, 53, 54, 55}
5
{57, 58, 59, 60, 61}
(352997, 1)
------


100%|██████████| 54/54 [18:11<00:00, 20.20s/it]


In [38]:
for shop_num in tqdm(range(1,55)):
    fn = '../cache/shifted_rs_df_shop1{}.csv'.format(shop_num)
    df1 = pd.read_csv(fn)

#     print(df1.shape)
#     print(set(df1.date_block_num))
#     df1 = df1.drop('my_id', axis=1)
    # forward
    # 1 month
    shifted_df = df1[['date_block_num','item_nbr','unit_sales_median']].copy()
#     print(shifted_df.shape)
    org_date_blocks = shifted_df.date_block_num.values
    shifted_df.date_block_num += 12
    shifted_df.columns = ['date_block_num','item_nbr', 'unit_sales_median_f_lag_12']
#     print(shifted_df.date_block_num.nunique())
    
    df1 = df1.merge(shifted_df, on=['date_block_num','item_nbr'], how='left').fillna(0.0)
    df1 = df1.drop_duplicates()
    
    assert(len(df1) == len(shifted_df))
    
    df = pd.DataFrame()
    df['id'] = df1.id.values
    df['unit_sales_median_f_lag_12'] = df1.unit_sales_median_f_lag_12.values
    df = df.loc[:, (df != 0).any(axis=0)]
    if len(df.columns) != 2:
        print(shop_num)
        print(df1.shape)
        print(set(df1.date_block_num))
        print(shifted_df.date_block_num.nunique())
        print(set(shifted_df.date_block_num))
        print(df.shape)
        print('------')
    fn = '../cache/lag12_sales_median_rs_df_shop{}.csv'.format(shop_num)
    delete_file_if_exists(fn)
    df.to_csv(fn, float_format='%.5f', index=False)
    del shifted_df
    del df1
    del df

 94%|█████████▍| 51/54 [16:01<00:56, 18.86s/it]

52
(352997, 8)
{51, 52, 53, 54, 55}
5
{64, 65, 66, 67, 63}
(352997, 1)
------


100%|██████████| 54/54 [16:28<00:00, 18.31s/it]


In [39]:
for shop_num in tqdm(range(1,55)):
    fn = '../cache/shifted_rs_df_shop1{}.csv'.format(shop_num)
    df1 = pd.read_csv(fn)

#     print(df1.shape)
#     print(set(df1.date_block_num))
#     df1 = df1.drop('my_id', axis=1)
    # forward
    # 1 month
    shifted_df = df1[['date_block_num','item_nbr','unit_sales_median']].copy()
#     print(shifted_df.shape)
    org_date_blocks = shifted_df.date_block_num.values
    shifted_df.date_block_num += 24
    shifted_df.columns = ['date_block_num','item_nbr', 'unit_sales_median_f_lag_24']
#     print(shifted_df.date_block_num.nunique())
    
    df1 = df1.merge(shifted_df, on=['date_block_num','item_nbr'], how='left').fillna(0.0)
    df1 = df1.drop_duplicates()
    
    assert(len(df1) == len(shifted_df))
    
    df = pd.DataFrame()
    df['id'] = df1.id.values
    df['unit_sales_median_f_lag_24'] = df1.unit_sales_median_f_lag_24.values
    df = df.loc[:, (df != 0).any(axis=0)]
    if len(df.columns) != 2:
        print(shop_num)
        print(df1.shape)
        print(set(df1.date_block_num))
        print(shifted_df.date_block_num.nunique())
        print(set(shifted_df.date_block_num))
        print(df.shape)
        print('------')
    fn = '../cache/lag24_sales_median_rs_df_shop{}.csv'.format(shop_num)
    delete_file_if_exists(fn)
    df.to_csv(fn, float_format='%.5f', index=False)
    del shifted_df
    del df1
    del df

 39%|███▉      | 21/54 [05:12<08:10, 14.87s/it]

22
(985899, 8)
{33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55}
23
{57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79}
(985899, 1)
------


 94%|█████████▍| 51/54 [12:54<00:45, 15.18s/it]

52
(352997, 8)
{51, 52, 53, 54, 55}
5
{75, 76, 77, 78, 79}
(352997, 1)
------


100%|██████████| 54/54 [13:15<00:00, 14.73s/it]


In [40]:
for shop_num in tqdm(range(1,55)):
    fn = '../cache/shifted_rs_df_shop1{}.csv'.format(shop_num)
    df1 = pd.read_csv(fn)

#     print(df1.shape)
#     print(set(df1.date_block_num))
#     df1 = df1.drop('my_id', axis=1)
    # forward
    # 1 month
    shifted_df = df1[['date_block_num','item_nbr','unit_sales_std']].copy()
#     print(shifted_df.shape)
    org_date_blocks = shifted_df.date_block_num.values
    shifted_df.date_block_num += 1
    shifted_df.columns = ['date_block_num','item_nbr', 'unit_sales_std_f_lag_1']
#     print(shifted_df.date_block_num.nunique())
    
    df1 = df1.merge(shifted_df, on=['date_block_num','item_nbr'], how='left').fillna(0.0)
    df1 = df1.drop_duplicates()
    
    assert(len(df1) == len(shifted_df))
    
    df = pd.DataFrame()
    df['id'] = df1.id.values
    df['unit_sales_std_f_lag_1'] = df1.unit_sales_std_f_lag_1.values
    df = df.loc[:, (df != 0).any(axis=0)]
    if len(df.columns) != 2:
        print(shop_num)
        print(df1.shape)
        print(set(df1.date_block_num))
        print(shifted_df.date_block_num.nunique())
        print(set(shifted_df.date_block_num))
        print(df.shape)
        print('------')
    fn = '../cache/lag1_sales_std_rs_df_shop{}.csv'.format(shop_num)
    delete_file_if_exists(fn)
    df.to_csv(fn, float_format='%.5f', index=False)
    del shifted_df
    del df1
    del df

100%|██████████| 54/54 [20:14<00:00, 22.50s/it]


In [41]:
for shop_num in tqdm(range(1,55)):
    fn = '../cache/shifted_rs_df_shop1{}.csv'.format(shop_num)
    df1 = pd.read_csv(fn)

#     print(df1.shape)
#     print(set(df1.date_block_num))
#     df1 = df1.drop('my_id', axis=1)
    # forward
    # 1 month
    shifted_df = df1[['date_block_num','item_nbr','unit_sales_std']].copy()
#     print(shifted_df.shape)
    org_date_blocks = shifted_df.date_block_num.values
    shifted_df.date_block_num += 3
    shifted_df.columns = ['date_block_num','item_nbr', 'unit_sales_std_f_lag_3']
#     print(shifted_df.date_block_num.nunique())
    
    df1 = df1.merge(shifted_df, on=['date_block_num','item_nbr'], how='left').fillna(0.0)
    df1 = df1.drop_duplicates()
    
    assert(len(df1) == len(shifted_df))
    
    df = pd.DataFrame()
    df['id'] = df1.id.values
    df['unit_sales_std_f_lag_3'] = df1.unit_sales_std_f_lag_3.values
    df = df.loc[:, (df != 0).any(axis=0)]
    if len(df.columns) != 2:
        print(shop_num)
        print(df1.shape)
        print(set(df1.date_block_num))
        print(shifted_df.date_block_num.nunique())
        print(set(shifted_df.date_block_num))
        print(df.shape)
        print('------')
    fn = '../cache/lag3_sales_std_rs_df_shop{}.csv'.format(shop_num)
    delete_file_if_exists(fn)
    df.to_csv(fn, float_format='%.5f', index=False)
    del shifted_df
    del df1
    del df

100%|██████████| 54/54 [19:21<00:00, 21.51s/it]


In [42]:
for shop_num in tqdm(range(1,55)):
    fn = '../cache/shifted_rs_df_shop1{}.csv'.format(shop_num)
    df1 = pd.read_csv(fn)

#     print(df1.shape)
#     print(set(df1.date_block_num))
#     df1 = df1.drop('my_id', axis=1)
    # forward
    # 1 month
    shifted_df = df1[['date_block_num','item_nbr','unit_sales_std']].copy()
#     print(shifted_df.shape)
    org_date_blocks = shifted_df.date_block_num.values
    shifted_df.date_block_num += 6
    shifted_df.columns = ['date_block_num','item_nbr', 'unit_sales_std_f_lag_6']
#     print(shifted_df.date_block_num.nunique())
    
    df1 = df1.merge(shifted_df, on=['date_block_num','item_nbr'], how='left').fillna(0.0)
    df1 = df1.drop_duplicates()
    
    assert(len(df1) == len(shifted_df))
    
    df = pd.DataFrame()
    df['id'] = df1.id.values
    df['unit_sales_std_f_lag_6'] = df1.unit_sales_std_f_lag_6.values
    df = df.loc[:, (df != 0).any(axis=0)]
    if len(df.columns) != 2:
        print(shop_num)
        print(df1.shape)
        print(set(df1.date_block_num))
        print(shifted_df.date_block_num.nunique())
        print(set(shifted_df.date_block_num))
        print(df.shape)
        print('------')
    fn = '../cache/lag6_sales_std_rs_df_shop{}.csv'.format(shop_num)
    delete_file_if_exists(fn)
    df.to_csv(fn, float_format='%.5f', index=False)
    del shifted_df
    del df1
    del df

 94%|█████████▍| 51/54 [17:50<01:02, 20.98s/it]

52
(352997, 8)
{51, 52, 53, 54, 55}
5
{57, 58, 59, 60, 61}
(352997, 1)
------


100%|██████████| 54/54 [18:20<00:00, 20.38s/it]


In [43]:
for shop_num in tqdm(range(1,55)):
    fn = '../cache/shifted_rs_df_shop1{}.csv'.format(shop_num)
    df1 = pd.read_csv(fn)

#     print(df1.shape)
#     print(set(df1.date_block_num))
#     df1 = df1.drop('my_id', axis=1)
    # forward
    # 1 month
    shifted_df = df1[['date_block_num','item_nbr','unit_sales_std']].copy()
#     print(shifted_df.shape)
    org_date_blocks = shifted_df.date_block_num.values
    shifted_df.date_block_num += 12
    shifted_df.columns = ['date_block_num','item_nbr', 'unit_sales_std_f_lag_12']
#     print(shifted_df.date_block_num.nunique())
    
    df1 = df1.merge(shifted_df, on=['date_block_num','item_nbr'], how='left').fillna(0.0)
    df1 = df1.drop_duplicates()
    
    assert(len(df1) == len(shifted_df))
    
    df = pd.DataFrame()
    df['id'] = df1.id.values
    df['unit_sales_std_f_lag_12'] = df1.unit_sales_std_f_lag_12.values
    df = df.loc[:, (df != 0).any(axis=0)]
    if len(df.columns) != 2:
        print(shop_num)
        print(df1.shape)
        print(set(df1.date_block_num))
        print(shifted_df.date_block_num.nunique())
        print(set(shifted_df.date_block_num))
        print(df.shape)
        print('------')
    fn = '../cache/lag12_sales_std_rs_df_shop{}.csv'.format(shop_num)
    delete_file_if_exists(fn)
    df.to_csv(fn, float_format='%.5f', index=False)
    del shifted_df
    del df1
    del df

 94%|█████████▍| 51/54 [16:08<00:56, 18.98s/it]

52
(352997, 8)
{51, 52, 53, 54, 55}
5
{64, 65, 66, 67, 63}
(352997, 1)
------


100%|██████████| 54/54 [16:35<00:00, 18.44s/it]


In [44]:
for shop_num in tqdm(range(1,55)):
    fn = '../cache/shifted_rs_df_shop1{}.csv'.format(shop_num)
    df1 = pd.read_csv(fn)

#     print(df1.shape)
#     print(set(df1.date_block_num))
#     df1 = df1.drop('my_id', axis=1)
    # forward
    # 1 month
    shifted_df = df1[['date_block_num','item_nbr','unit_sales_std']].copy()
#     print(shifted_df.shape)
    org_date_blocks = shifted_df.date_block_num.values
    shifted_df.date_block_num += 24
    shifted_df.columns = ['date_block_num','item_nbr', 'unit_sales_std_f_lag_24']
#     print(shifted_df.date_block_num.nunique())
    
    df1 = df1.merge(shifted_df, on=['date_block_num','item_nbr'], how='left').fillna(0.0)
    df1 = df1.drop_duplicates()
    
    assert(len(df1) == len(shifted_df))
    
    df = pd.DataFrame()
    df['id'] = df1.id.values
    df['unit_sales_std_f_lag_24'] = df1.unit_sales_std_f_lag_24.values
    df = df.loc[:, (df != 0).any(axis=0)]
    if len(df.columns) != 2:
        print(shop_num)
        print(df1.shape)
        print(set(df1.date_block_num))
        print(shifted_df.date_block_num.nunique())
        print(set(shifted_df.date_block_num))
        print(df.shape)
        print('------')
    fn = '../cache/lag24_sales_std_rs_df_shop{}.csv'.format(shop_num)
    delete_file_if_exists(fn)
    df.to_csv(fn, float_format='%.5f', index=False)
    del shifted_df
    del df1
    del df

 39%|███▉      | 21/54 [05:13<08:12, 14.91s/it]

22
(985899, 8)
{33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55}
23
{57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79}
(985899, 1)
------


 94%|█████████▍| 51/54 [12:55<00:45, 15.21s/it]

52
(352997, 8)
{51, 52, 53, 54, 55}
5
{75, 76, 77, 78, 79}
(352997, 1)
------


100%|██████████| 54/54 [13:17<00:00, 14.77s/it]


In [3]:
for shop_num in tqdm(range(22,55)):
    df1 = pd.DataFrame()
    
    df1['unit_sales_mean_f_lag_1'] =0
    df1['unit_sales_mean_f_lag_3'] = 0
    df1['unit_sales_mean_f_lag_6'] = 0
    df1['unit_sales_mean_f_lag_12'] = 0
    df1['unit_sales_mean_f_lag_24'] = 0
    
    df1['unit_sales_median_f_lag_1'] =0
    df1['unit_sales_median_f_lag_3'] = 0
    df1['unit_sales_median_f_lag_6'] = 0
    df1['unit_sales_median_f_lag_12'] = 0
    df1['unit_sales_median_f_lag_24'] = 0
    
    df1['unit_sales_std_f_lag_1'] =0
    df1['unit_sales_std_f_lag_3'] = 0
    df1['unit_sales_std_f_lag_6'] = 0
    df1['unit_sales_std_f_lag_12'] = 0
    df1['unit_sales_std_f_lag_24'] = 0
    
    fn = '../cache/lag1_sales_mean_rs_df_shop{}.csv'.format(shop_num)
    df2 = pd.read_csv(fn)
    
    fn = '../cache/lag3_sales_mean_rs_df_shop{}.csv'.format(shop_num)
    df3 = pd.read_csv(fn)
    
    fn = '../cache/lag6_sales_mean_rs_df_shop{}.csv'.format(shop_num)
    df4 = pd.read_csv(fn)
    
    fn = '../cache/lag12_sales_mean_rs_df_shop{}.csv'.format(shop_num)
    df5 = pd.read_csv(fn)
    
    fn = '../cache/lag24_sales_mean_rs_df_shop{}.csv'.format(shop_num)
    df6 = pd.read_csv(fn)
    print('1..')
    df1['id'] = df2['id'].values
    if len(df2.columns) > 1:
        df1['unit_sales_mean_f_lag_1'] = df2.unit_sales_mean_f_lag_1.values
        
    if len(df3.columns) > 1:
        df1['unit_sales_mean_f_lag_3'] = df3.unit_sales_mean_f_lag_3.values
        
        
    if len(df4.columns) > 1:
        df1['unit_sales_mean_f_lag_6'] = df4.unit_sales_mean_f_lag_6.values
        
    if len(df5.columns) > 1:
        df1['unit_sales_mean_f_lag_12'] = df5.unit_sales_mean_f_lag_12.values
        
    if len(df6.columns) > 1:
        df1['unit_sales_mean_f_lag_24'] = df6.unit_sales_mean_f_lag_24.values
     
    del df2, df3, df4, df5, df6
    print('2..')
    fn = '../cache/lag1_sales_median_rs_df_shop{}.csv'.format(shop_num)
    df2 = pd.read_csv(fn)
    
    fn = '../cache/lag3_sales_median_rs_df_shop{}.csv'.format(shop_num)
    df3 = pd.read_csv(fn)
    
    fn = '../cache/lag6_sales_median_rs_df_shop{}.csv'.format(shop_num)
    df4 = pd.read_csv(fn)
    
    fn = '../cache/lag12_sales_median_rs_df_shop{}.csv'.format(shop_num)
    df5 = pd.read_csv(fn)
    
    fn = '../cache/lag24_sales_median_rs_df_shop{}.csv'.format(shop_num)
    df6 = pd.read_csv(fn)
    print('3..')
#     df1['id'] = df2['id'].values
    if len(df2.columns) > 1:
        df1['unit_sales_median_f_lag_1'] = df2.unit_sales_median_f_lag_1.values
        
    if len(df3.columns) > 1:
        df1['unit_sales_median_f_lag_3'] = df3.unit_sales_median_f_lag_3.values
        
        
    if len(df4.columns) > 1:
        df1['unit_sales_median_f_lag_6'] = df4.unit_sales_median_f_lag_6.values
        
    if len(df5.columns) > 1:
        df1['unit_sales_median_f_lag_12'] = df5.unit_sales_median_f_lag_12.values
        
    if len(df6.columns) > 1:
        df1['unit_sales_median_f_lag_24'] = df6.unit_sales_median_f_lag_24.values
     
    del df2, df3, df4, df5, df6
    print('4..')
    fn = '../cache/lag1_sales_std_rs_df_shop{}.csv'.format(shop_num)
    df2 = pd.read_csv(fn)
    
    fn = '../cache/lag3_sales_std_rs_df_shop{}.csv'.format(shop_num)
    df3 = pd.read_csv(fn)
    
    fn = '../cache/lag6_sales_std_rs_df_shop{}.csv'.format(shop_num)
    df4 = pd.read_csv(fn)
    
    fn = '../cache/lag12_sales_std_rs_df_shop{}.csv'.format(shop_num)
    df5 = pd.read_csv(fn)
    
    fn = '../cache/lag24_sales_std_rs_df_shop{}.csv'.format(shop_num)
    df6 = pd.read_csv(fn)
    
#     df1['id'] = df2['id'].values
    if len(df2.columns) > 1:
        df1['unit_sales_std_f_lag_1'] = df2.unit_sales_std_f_lag_1.values
        
    if len(df3.columns) > 1:
        df1['unit_sales_std_f_lag_3'] = df3.unit_sales_std_f_lag_3.values
        
        
    if len(df4.columns) > 1:
        df1['unit_sales_std_f_lag_6'] = df4.unit_sales_std_f_lag_6.values
        
    if len(df5.columns) > 1:
        df1['unit_sales_std_f_lag_12'] = df5.unit_sales_std_f_lag_12.values
        
    if len(df6.columns) > 1:
        df1['unit_sales_std_f_lag_24'] = df6.unit_sales_std_f_lag_24.values
        
    fn = '../cache/df_all_lag_shop{}.csv'.format(shop_num)
    delete_file_if_exists(fn)
    
#     const_col = []
#     for col in df1.columns:
#         c = len(set(df1[col].values))
#         if c==1:
#             const_col.append(col)
#     if len(const_col) > 0:
#         c = len(const_col)
#         print(shop_num)
#         print(c)
#         print(df1.shape)
#         print(const_col)
    df1.to_csv(fn, float_format='%.5f', index=False)
    del df1
    del df2, df3, df4, df5, df6
    print('5..')
    gc.collect()

  0%|          | 0/33 [00:00<?, ?it/s]

1..
2..
3..
4..


  3%|▎         | 1/33 [00:20<11:05, 20.80s/it]

5..
1..
2..
3..
4..


  6%|▌         | 2/33 [01:21<21:06, 40.87s/it]

...

5..
1..
2..
3..
4..


 97%|█████████▋| 32/33 [32:47<01:01, 61.49s/it]

5..
1..
2..
3..
4..


100%|██████████| 33/33 [33:31<00:00, 60.96s/it]

5..


In [4]:
for shop_num in range(1,55):
    fn = '../cache/df_all_lag_shop{}.csv'.format(shop_num)
    df = pd.read_csv(fn)
    fn1 = '../cache/df_all_rs_lag_shop{}.csv'.format(shop_num)
    df.to_csv(fn1, float_format='%.5f', index=False)
    delete_file_if_exists(fn)
    del df

In [3]:
for shop_num in tqdm(range(1,55)):
    fn_org = '../cache/bkup/train10_t_store{}.csv'.format(shop_num)
    fn_org1 = '../cache/bkup2/test7_1_t_store{}.csv'.format(shop_num)
    print(fn_org1)
    train_org = pd.read_csv(fn_org)
    test_org = pd.read_csv(fn_org1)
    df_shop_all = pd.concat([train_org, test_org], axis=0)
    
#     fn_org = '../cache/bkup/train10_t_store{}.csv'.format(shop_num)
#     train_org = pd.read_csv(fn_org)
    
    fn = '../cache/df_all_rs_lag_shop{}.csv'.format(shop_num)
    df = pd.read_csv(fn)
    df['date'] = df_shop_all.date.values
    
    df_train = df[:len(train_org)]
    df_train = df_train[(df_train.date.astype('str') >= str('2015-01-01')) & 
                        (df_train.date.astype('str') <= str('2017-08-15'))]
    df_train = df_train.drop('date', axis=1)
    
    df_test = df[len(train_org):]
    df_test = df_test.drop('date', axis=1)
    assert(len(test_org) == len(df_test))
    
    fn_tr = '../cache/df_train10_t_rs_lag_shop{}.csv'.format(shop_num)
    fn_te = '../cache/df_test7_1_rs_lag_shop{}.csv'.format(shop_num)
    delete_file_if_exists(fn_tr)
    delete_file_if_exists(fn_te)
    
    df_train.to_csv(fn_tr, index=False, float_format='%.5f')
    df_test.to_csv(fn_te, index=False, float_format='%.5f')
    
    del df_shop_all
    del train_org, test_org
    del df_train
    del df_test

  0%|          | 0/54 [00:00<?, ?it/s]

../cache/bkup2/test7_1_t_store1.csv


/home/watts/anaconda2/envs/sdp35/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
  2%|▏         | 1/54 [01:52<1:39:26, 112.57s/it]

../cache/bkup2/test7_1_t_store2.csv


  4%|▎         | 2/54 [03:59<1:43:55, 119.91s/it]

...

 96%|█████████▋| 52/54 [1:35:53<03:41, 110.64s/it]

../cache/bkup2/test7_1_t_store53.csv


 98%|█████████▊| 53/54 [1:37:41<01:50, 110.59s/it]

../cache/bkup2/test7_1_t_store54.csv


100%|██████████| 54/54 [1:38:58<00:00, 109.98s/it]


In [10]:
for i in range(1,55):
    process_test_csv_file(i) 

../cache/bkup2/test7_1_t_store1.csv


/home/watts/anaconda2/envs/sdp35/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2850: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):
100%|██████████| 16/16 [01:38<00:00,  6.16s/it]


(2624569, 28)
(62416, 28)
(16, 31)
0
56
../cache/test7_1_t_store1_recent_stats_1.csv
(62416, 57)
../cache/bkup2/test7_1_t_store2.csv


100%|██████████| 16/16 [01:54<00:00,  7.17s/it]


(3050256, 28)
(62416, 28)
(16, 31)
0
56
../cache/test7_1_t_store2_recent_stats_1.csv
(62416, 57)
../cache/bkup2/test7_1_t_store3.csv


...

(62416, 28)
(16, 31)
0
38
../cache/test7_1_t_store52_recent_stats_1.csv
(62416, 57)
../cache/bkup2/test7_1_t_store53.csv


100%|██████████| 16/16 [01:15<00:00,  4.74s/it]


(2000671, 28)
(62416, 28)
(16, 31)
0
56
../cache/test7_1_t_store53_recent_stats_1.csv
(62416, 57)
../cache/bkup2/test7_1_t_store54.csv


100%|██████████| 16/16 [01:04<00:00,  4.02s/it]


(1711283, 28)
(62416, 28)
(16, 31)
0
56
../cache/test7_1_t_store54_recent_stats_1.csv
(62416, 57)


In [3]:
from scipy.sparse import save_npz, load_npz
import sys

sys.stdout = open('/dev/stdout', 'w')
new_cols = []

In [4]:
recent_days = [30, 90, 180, 365, 730]
for days in tqdm(recent_days):
    new_cols.append('us_{}_mean'.format(days))
    new_cols.append('us_{}_med'.format(days))
    new_cols.append('us_{}_std'.format(days))
    new_cols.append('us_{}_skew'.format(days))
    new_cols.append('us_{}_kurt'.format(days))
    new_cols.append('us_{}_tbn'.format(days))

100%|██████████| 5/5 [00:00<00:00, 33500.83it/s]


In [5]:
df_test_rs = pd.DataFrame()
for store_num in tqdm(range(1,55)):
    fn = '../cache/test7_1_t_store{}_recent_stats_1.csv'.format(store_num)
    df = pd.read_csv(fn)
    df_test_rs = pd.concat([df_test_rs, df], axis=0)
    del df


100%|██████████| 54/54 [01:05<00:00,  1.21s/it]


In [6]:
df_test_rs = df_test_rs.sort_values('id')
df_test_rs = df_test_rs[new_cols]

In [7]:
df_test_rs.shape

(3370464, 30)

In [8]:
delete_file_if_exists('../cache/df_test_rs.csv')
df_test_rs.to_csv('../cache/df_test_rs.csv', index=False, float_format='%.5f')

In [9]:
X_test_rs = df_test_rs.values
np.save('../cache/X_test_rs', X_test_rs)

In [ ]:
# process_csv_file(1)

In [ ]:
# Multi-threaded processes to utilize all available CPUs for this task. Note that many threads will block on IO
# so creating more than number of CPUs.    
# thread_pool = Pool(32)
# thread_pool.map (segment_pad_and_save_ct_scan_as_npz, dicom_folder_list)

# # Cleanup
# thread_pool.close()
# thread_pool.join_thread()

In [ ]:
gc.collect()

In [ ]:
# i = 1
# fn = '../cache/train10_t_store{}.csv'.format(i)
# df = pd.read_csv(fn)
# df.unit_sales[df.unit_sales < 0] = 0
# df1 = df.set_index('date')
# dates = np.unique(df.date.values)
# for j, date in tqdm(enumerate(dates)):
#     if j > 0:
#         break
#     print(j)
#     calculate_recent_stats(date, df, df1)
# for days in [15, 30, 60, 90, 180, 365, 730, 1095, 1460]:
#     df = set_recent_df_stats(df ,df1, days)
# print(df.shape)
# print(sum(df.isnull().any()==True))
# print(sum((df != 0).any(axis=0)))
# fn = '../cache/train10_t_store{}_recent_stats.csv'.format(i)
# print(fn)

In [ ]:
# df.isnull().sum()

In [ ]:
# for i in tqdm(range(1,55)):
#     fn_tmp = '../cache/train10_t_store{}_recent_stats_tmp.csv'.format(i)
#     delete_file_if_exists(fn_tmp)
#     fn = '../cache/train10_t_store{}_recent_stats.csv'.format(i)
#     df = pd.read_csv(fn)
#     df.to_csv(fn, float_format='%.5f', index=False)
#     del df